In [ ]:
import sys, pickle, re, string, json, nltk

import pandas as pd
import numpy as np
import copy as cp

from os import listdir
from os.path import isfile, join
from tqdm import tqdm
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
FOLDER_PATH = '/content/drive/MyDrive/Knowledge Graphs 2022/'
%cd $FOLDER_PATH

/content/drive/.shortcut-targets-by-id/18wZgIc4f2VuuQ34uv_vRrBAOHvMWSEaq/Knowledge Graphs 2022


## 1. Data loading

##### json files to tweet2data dict

In [ ]:
## only need to run once

# f = listdir("data/user_tweets/")
# all_tweets = dict()
# for filename in tqdm(f):
#   posts = []

#   try:
#     with open(path+filename, 'r') as file:
#       data = file.read().replace('}{', '},{')
#       data = '[' + data + ']'
#       post = json.loads(data)
#       posts.append(post)

#     all_tweets[filename] = posts
#   except:
#     print(filename)

## CHECKPOINT
# tweets_pickle_file = 'data/all_tweets.pickle'
# with open(tweets_pickle_file, 'wb') as tweets_pickle:
#   pickle.dump(all_tweets, tweets_pickle, protocol=pickle.HIGHEST_PROTOCOL)

# with open(tweets_pickle_file, 'rb') as tweets_pickle:
#   test = pickle.load(tweets_pickle)

# print(all_tweets == test)

In [ ]:
tweets_pickle_file = 'data/all_tweets.pickle'
with open(tweets_pickle_file, 'rb') as tweets_pickle:
   all_tweets = pickle.load(tweets_pickle)

In [ ]:
num_tweets = 0
for k in all_tweets.keys():
  num_tweets += len(all_tweets[k][0])
num_tweets

956028

##### tweet2data dict to structured tweet representation

In [ ]:
# class Tweet:
#   def __init__(self, id, text, time, user_id, user_screen_name):
#     self.id = id
#     self.text = text
#     self.time = time
#     self.user_id = user_id
#     self.user_screen_name = user_screen_name
  
#   def to_dict(self):
#     return {'id': self.id, 'text': self.text, 'time': self.time, 'user_id': self.user_id, 'user_screen_name': self.user_screen_name}

In [ ]:
## only need to run once

# only needs to run once
# corrupted_data_list = []
# tweets_list = []

# for t in all_tweets.keys():
#   try:
#     user_tweets = all_tweets[t][0]
#     for user_tweet in user_tweets: 
#       tweet = Tweet(user_tweet['id'], user_tweet['full_text'], user_tweet['created_at'], user_tweet['user']['id'], user_tweet['user']['screen_name'])
#       tweets_list.append(tweet.to_dict())
#   except:
#     print(f"{t}")
#     corrupted_data_list.append(t)

## CHECKPOINT
# tweets_df = pd.DataFrame(tweets_list)
# tweets_df.to_csv("topic modeling/tweets_w_user_info.csv", index=False)

In [ ]:
tweets_df = pd.read_csv("topic modeling/tweets_w_user_info.csv")
tweets_df

,id,text,time,user_id,user_screen_name
0,1420959115879145474,https://t.co/3bM9PoSxS6 #acoustic #orchestra #...,Fri Jul 30 04:07:08 +0000 2021,1404563992907976706,teknoaxe
1,1420589578541428739,@DekeSlade Hey Rj! A link to my youtube chann...,Thu Jul 29 03:38:43 +0000 2021,1404563992907976706,teknoaxe
2,1420533067400105985,@kraftoki @MinxMore How do you know?,Wed Jul 28 23:54:10 +0000 2021,1404563992907976706,teknoaxe
3,1420373718828679174,"@MinxMore Also, consider that ""immune for only...",Wed Jul 28 13:20:58 +0000 2021,1404563992907976706,teknoaxe
4,1420373378653773829,@MinxMore It's like chicken pox....,Wed Jul 28 13:19:37 +0000 2021,1404563992907976706,teknoaxe
...,...,...,...,...,...
956023,1397974274367963136,@Baklava_in_CA @jordanbpeterson I got my pound.,Thu May 27 17:53:34 +0000 2021,218741203,mml111mml111
956024,1397972983067693064,@nypost He tries to turn his hotel rooms into ...,Thu May 27 17:48:26 +0000 2021,218741203,mml111mml111
956025,1397972391708475392,@Baklava_in_CA @jordanbpeterson I've realized ...,Thu May 27 17:46:05 +0000 2021,218741203,mml111mml111
956026,1397813255523995648,"So, what do you think the long term side effec...",Thu May 27 07:13:44 +0000 2021,218741203,mml111mml111


##### tweet2text (id, text, time)

In [ ]:
## only needs to run once

# corrupted_data_list = []
# twitter_raw_text_dict = dict()
# twitter_raw_text_dict['id'] = []
# twitter_raw_text_dict['text'] = []
# twitter_raw_text_dict['time'] = []

# for t in all_tweets.keys():
#   try:
#     user_tweets = all_tweets[t][0]
#     for user_tweet in user_tweets: 
#       twitter_raw_text_dict['id'].append(user_tweet['id'])
#       twitter_raw_text_dict['text'].append(user_tweet['full_text'])
#       twitter_raw_text_dict['time'].append(user_tweet['created_at'])
#   except:
#     print(f"{t}")
#     corrupted_data_list.append(t)
# print(len(twitter_raw_text_dict))
# print(len(all_tweets))
# print(len(corrupted_data_list))

# tweets_pickle_file = 'topic modeling/tweets_raw_text.pickle'

## CHECKPOINT
# with open(tweets_pickle_file, 'wb') as tweets_pickle:
#   pickle.dump(twitter_raw_text_dict, tweets_pickle, protocol=pickle.HIGHEST_PROTOCOL)

# with open(tweets_pickle_file, 'rb') as tweets_pickle:
#   test = pickle.load(tweets_pickle)

# print(twitter_raw_text_dict == test)

In [ ]:
tweets_pickle_file = 'topic modeling/tweets_raw_text.pickle'

with open(tweets_pickle_file, 'rb') as tweets_pickle:
  twitter_raw_text_dict = pickle.load(tweets_pickle)

print(len(twitter_raw_text_dict))

3


## 2. Text Cleaning


In [ ]:
df = pd.DataFrame.from_dict(twitter_raw_text_dict)
df
# df.to_csv("topic modeling/all_tweets_w_time.csv", index=False)

,id,text,time
0,1420959115879145474,https://t.co/3bM9PoSxS6 #acoustic #orchestra #...,Fri Jul 30 04:07:08 +0000 2021
1,1420589578541428739,@DekeSlade Hey Rj! A link to my youtube chann...,Thu Jul 29 03:38:43 +0000 2021
2,1420533067400105985,@kraftoki @MinxMore How do you know?,Wed Jul 28 23:54:10 +0000 2021
3,1420373718828679174,"@MinxMore Also, consider that ""immune for only...",Wed Jul 28 13:20:58 +0000 2021
4,1420373378653773829,@MinxMore It's like chicken pox....,Wed Jul 28 13:19:37 +0000 2021
...,...,...,...
956023,1397974274367963136,@Baklava_in_CA @jordanbpeterson I got my pound.,Thu May 27 17:53:34 +0000 2021
956024,1397972983067693064,@nypost He tries to turn his hotel rooms into ...,Thu May 27 17:48:26 +0000 2021
956025,1397972391708475392,@Baklava_in_CA @jordanbpeterson I've realized ...,Thu May 27 17:46:05 +0000 2021
956026,1397813255523995648,"So, what do you think the long term side effec...",Thu May 27 07:13:44 +0000 2021


##### keywords representation of text

In [ ]:
# initialize wordset
WORDSET = set()
# download stopwords list from nltk library
stop_words_en = stopwords.words('english')
stop_words_spa = stopwords.words('spanish')
stop_words_french = stopwords.words('french')
stop_words_german = stopwords.words('german')
stop_words_dutch = stopwords.words('dutch')
# manually extend stopwards list
stop_words_en.extend(["i'm", "\n", "amp", "it's", "it’s", "im", "via", "like", "day", 
                  "time", "go", "got", "get", "one", "lol", "y'all", "y’all", "gonna", "fuck", 
                  "shit", "pls", "plz", "tbh", "rn", "tbt", "ngl", "idc", "idk", "imy", 
                    "i'll", "i've", "let's", "jk", "cuz", "fucking", "fuckin" "wow", "that's", 
                   "dawg", "drop", "pin", "i'd", "lmao", "wtf", "link", "bio", "amp", "rt"])

"""
Helper function that removes emoji characters from a given text. 

Parameters: 
  string -- string of text 
Returns: 
  demoji_string -- text without emoji characters

"""
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
 
    demoji_string = emoji_pattern.sub(r'', str(string))
    return demoji_string
    

"""
Helper function that transforms part of speech tag to wordnet tag. 

Parameters: 
  nltk_tag -- nltk part of speech tag
Returns: 
  corresponding wordnet tag
"""
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

"""
Cleans the text of the tweet by removing non-alphanumeric characters and 
stop words, and lemmatizing and tokenizing text. 

Parameters: 
  text -- string of tweet text 

Returns: 
  new_text -- cleaned and tokenized text
"""
def clean_text(text):
  # Remove emojis
  text = remove_emoji(text)
  # Remove urls
  text = re.sub(r"https\S+", "", text)
  # Remove mentions
  text = re.sub(r"@\S+", "", text)
  # Remove hashtags
  text = re.sub(r"#\S+", "", text)
  # Remove numbers
  text = re.sub(r"[0-9\-]", "", text)
  # Remove punctuation 
  text = re.sub('[,\/.!?:;_#\-\"\“@&%*($)+]', "",text)
  text = re.sub('\’',"\'", text)
  # Remove non-alphabets
  text = re.sub(r'[^a-zA-Z0-9,\/.!?:; ]', "",text)
  # Lower case letters 
  text = text.lower()

  # Part of speech tagging
  pos_tag = nltk.pos_tag(text.split())
  wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), pos_tag)
  # Lemmatize
  lemmatizer = WordNetLemmatizer() 
  #stemmer = SnowballStemmer("english") 
  lemmatized_words = []
  
  for word, tag in wordnet_tagged:
      if tag is None: 
        lword = lemmatizer.lemmatize(word)
      else:
        lword = lemmatizer.lemmatize(word, pos=tag)
      #sword = stemmer.stem(lword)
      if (lword not in stop_words_en) and (lword not in stop_words_spa) and \
      (lword not in stop_words_french) and (lword not in stop_words_german) and \
      (lword not in stop_words_dutch) and (len(lword) > 3):
        lemmatized_words.append(lword)
        WORDSET.add(lword)
      

  new_text = lemmatized_words
  
  return new_text

connected_df_dict = df.to_dict(orient = 'records')
total = len(connected_df_dict)
pbar2 = tqdm(total=total, position = 0, leave = True)

cleaned_tweets = cp.copy(connected_df_dict)
for tweet in cleaned_tweets:
  tweet['cleaned_text'] = clean_text(cp.copy(tweet['text']))
  pbar2.update(1)


pbar2.close() 

100%|██████████| 956028/956028 [15:37<00:00, 1019.48it/s]


In [ ]:
cleaned_tweets_df = pd.DataFrame(cleaned_tweets)
cleaned_tweets_df.drop_duplicates(subset='id', inplace=True)
cleaned_tweets_df.to_csv("data/tweets_cleaned_text.csv", index=False)
cleaned_tweets_df

,id,text,time,cleaned_text
0,1420959115879145474,https://t.co/3bM9PoSxS6 #acoustic #orchestra #...,Fri Jul 30 04:07:08 +0000 2021,[]
1,1420589578541428739,@DekeSlade Hey Rj! A link to my youtube chann...,Thu Jul 29 03:38:43 +0000 2021,"[youtube, channel, video, music, feature, good..."
2,1420533067400105985,@kraftoki @MinxMore How do you know?,Wed Jul 28 23:54:10 +0000 2021,[know]
3,1420373718828679174,"@MinxMore Also, consider that ""immune for only...",Wed Jul 28 13:20:58 +0000 2021,"[consider, immune, month, come, journalist, po..."
4,1420373378653773829,@MinxMore It's like chicken pox....,Wed Jul 28 13:19:37 +0000 2021,[chicken]
...,...,...,...,...
956023,1397974274367963136,@Baklava_in_CA @jordanbpeterson I got my pound.,Thu May 27 17:53:34 +0000 2021,[pound]
956024,1397972983067693064,@nypost He tries to turn his hotel rooms into ...,Thu May 27 17:48:26 +0000 2021,"[turn, hotel, room, vermont]"
956025,1397972391708475392,@Baklava_in_CA @jordanbpeterson I've realized ...,Thu May 27 17:46:05 +0000 2021,"[realize, never, debt, alone, interest, centra..."
956026,1397813255523995648,"So, what do you think the long term side effec...",Thu May 27 07:13:44 +0000 2021,"[think, long, term, side, effect]"


conservative text cleaning (preserve sentence structure)

In [ ]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
 
    demoji_string = emoji_pattern.sub(r'', str(string))
    return demoji_string

"""
Cleans the text of the tweet by removing non-alphanumeric characters and 
stop words, and lemmatizing and tokenizing text. 

Parameters: 
  text -- string of tweet text 

Returns: 
  new_text -- cleaned and tokenized text
"""
def clean_text(text):
  
  # Remove emojis
  text = remove_emoji(text)
  # Remove urls
  text = re.sub(r"https\S+", "", text)
  # Remove mentions
  text = re.sub(r"@\S+", "", text)
  # Remove hashtags
  text = re.sub(r"#\S+", "", text)
  # Remove symbols like "&amp"
  text = re.sub(r"&\S+", "", text)
  # Only keep alphabets
  text = re.sub(r'[^a-zA-Z0-9,\/.!?:; ]', "",text)
  text = text.lstrip()
  text = text.rstrip()
  
  return text

connected_df_dict = df.to_dict(orient = 'records')
total = len(connected_df_dict)
pbar2 = tqdm(total=total, position = 0, leave = True)

cleaned_tweets = cp.copy(connected_df_dict)
for tweet in cleaned_tweets:
  tweet['cleaned_text'] = clean_text(cp.copy(tweet['text']))
  pbar2.update(1)


pbar2.close() 

100%|██████████| 956028/956028 [00:13<00:00, 70043.80it/s]


In [ ]:
cleaned_tweets_df = pd.DataFrame(cleaned_tweets)
cleaned_tweets_df.drop_duplicates(subset='id', inplace=True)
tweets_df.drop_duplicates(subset='id', inplace=True)
tweets_df['cleaned_text'] = cleaned_tweets_df['cleaned_text']
tweets_df.to_csv("data/tweets_cleaned_sentence.csv", index=False)
tweets_df

,id,text,time,user_id,user_screen_name,cleaned_text
0,1420959115879145474,https://t.co/3bM9PoSxS6 #acoustic #orchestra #...,Fri Jul 30 04:07:08 +0000 2021,1404563992907976706,teknoaxe,
1,1420589578541428739,@DekeSlade Hey Rj! A link to my youtube chann...,Thu Jul 29 03:38:43 +0000 2021,1404563992907976706,teknoaxe,Hey Rj! A link to my youtube channel or the v...
2,1420533067400105985,@kraftoki @MinxMore How do you know?,Wed Jul 28 23:54:10 +0000 2021,1404563992907976706,teknoaxe,How do you know?
3,1420373718828679174,"@MinxMore Also, consider that ""immune for only...",Wed Jul 28 13:20:58 +0000 2021,1404563992907976706,teknoaxe,"Also, consider that immune for only a few mont..."
4,1420373378653773829,@MinxMore It's like chicken pox....,Wed Jul 28 13:19:37 +0000 2021,1404563992907976706,teknoaxe,Its like chicken pox....
...,...,...,...,...,...,...
956023,1397974274367963136,@Baklava_in_CA @jordanbpeterson I got my pound.,Thu May 27 17:53:34 +0000 2021,218741203,mml111mml111,I got my pound.
956024,1397972983067693064,@nypost He tries to turn his hotel rooms into ...,Thu May 27 17:48:26 +0000 2021,218741203,mml111mml111,He tries to turn his hotel rooms into Vermont?
956025,1397972391708475392,@Baklava_in_CA @jordanbpeterson I've realized ...,Thu May 27 17:46:05 +0000 2021,218741203,mml111mml111,Ive realized we are never going to pay that de...
956026,1397813255523995648,"So, what do you think the long term side effec...",Thu May 27 07:13:44 +0000 2021,218741203,mml111mml111,"So, what do you think the long term side effec..."
